In [11]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
import mantis_wave_modelling.finite_difference.parameter_handling as manFD

In [19]:
manFD.write_segy(filename = "text")

<module 'segyio.tools' from '/Users/gpapageo/opt/anaconda3/envs/mantis-core-examples/lib/python3.11/site-packages/segyio/tools.py'>


In [27]:
d = manFD.JsonParameterFile(filename= "./wave_modelling_backend/parameters.json")
d.validate_data()

Valid JSON


In [31]:
import numpy as np
import csv
d = {}
with open("./wave_modelling_backend/input.csv", newline='') as f:
    reader = csv.reader(f)
    headers = ["Top Depth", "Vp", "Vs", "Rho"]
    next(reader)
    for j, row in enumerate(reader):
        d[j] = {}
        for i, h in enumerate(row):
            d[j][headers[i]] = float(h)
models = {
    index: {"model": "generic", "parameters": {}}
    for index, _ in enumerate(d.keys())
}

In [36]:
grids = manFD.FiniteDifferenceGridFromRockPhysicsModels(well_input=d,  mod_dictionary=models, label="test")

In [ ]:
    with segyio.open(file, ignore_geometry=True) as f:
        # Get basic attributes
        n_traces = f.tracecount
        sample_rate = segyio.tools.dt(f) / 1000
        n_samples = f.samples.size
        twt = f.samples
        data = f.trace.raw[:]

In [ ]:
with segyio.open(temp_in, ignore_geometry=True) as src:
    spec = segyio.tools.metadata(src)
    spec.tracecount = len(indices)
    with segyio.create(temp_out, spec) as dst:
        for i in range(1 + src.ext_headers):
            dst.text[i] = src.text[i]
        dst.bin = src.bin
        dst.header = src.header

to do:
 - improve model scanning and error reporting
 - incorporate the parameter json as part of the grid making
 - enhance the calling function in the back end
 - fit viscoelastic and anisotropic models so that the generated grids are more accurate (easiest way to patch at this early stage is to use the thomsenVTI and tilt rotation classes).
 - improve the reporting on the file that does the export in the original code
 - create plot methods for the models/grids
 - create a method for saving the outputs to segy
 - create a method for getting the snapshots into a video files
 - write the examples and docstrings